In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser
from glob import glob
import pandas as pd
from os.path import join
from tifffile import TiffFile, imread
from ipywidgets import Text
import seaborn as sns
from pathlib import Path

## Select # of species, folders, and labels

In [2]:
numspeciestext = Text(description='# Species', value='2'); numspeciestext

Text(value='2', description='# Species')

In [3]:
# images folder
foldchoosers = [FileChooser() for i in range(int(numspeciestext.value))]
[display(foldchooser) for foldchooser in foldchoosers];

FileChooser(path='C:\Users\dnfst\Jupyter\multicolor roi drawing', filename='', title='HTML(value='', layout=La…

FileChooser(path='C:\Users\dnfst\Jupyter\multicolor roi drawing', filename='', title='HTML(value='', layout=La…

FileChooser(path='C:\Users\dnfst\Jupyter\multicolor roi drawing', filename='', title='HTML(value='', layout=La…

In [4]:
roipathtext = Text(description='RoI path:', value='combined images\masks'); roipathtext

Text(value='combined images\\masks', description='RoI path:')

In [5]:
default_labels = ['mEGFP', 'mEYFP', 'mCherry2']
labelstext = [Text(description='Species #'+str(i+1), value=default_labels[i]) \
              for i in range(int(numspeciestext.value))]
[display(labeltext) for labeltext in labelstext];

Text(value='mEGFP', description='Species #1')

Text(value='mEYFP', description='Species #2')

Text(value='mCherry2', description='Species #3')

## Calculate emission spectra (time consuming)

In [6]:
folders = [foldchooser.selected_path for foldchooser in foldchoosers]
labels = [labeltext.value for labeltext in labelstext]
spectra = []

for ifold, folder in enumerate(folders):
    files = glob(join(folder, '*.lsm'))
    if ifold == 0:
        image0_tif = TiffFile(files[0])
        channels = image0_tif.lsm_metadata['ChannelColors']['ColorNames']
        spectra.append(pd.DataFrame(data={'Channel': channels}))
    spectrum = np.zeros(len(channels))
    for fidx, file in enumerate(files):
        image = imread(file)[0]
        label = Path(file).stem[:2]
        roifiles = glob(join(folder, roipathtext.value, label+' *ROI??.npy'))
        for ridx, rf in enumerate(roifiles):
            roi = np.load(rf)
            spectrum += image.sum(
                           where=roi[:,None]*np.ones([1,len(channels),1,1], dtype='bool'), 
                           axis=(0,2,3))
    spectrum_normed = spectrum.astype('float') / spectrum.sum()
    spectra.append(pd.DataFrame(data={labels[ifold]: spectrum_normed}))
    
spectra_df = pd.concat(spectra, axis=1)

## Inspect; table and plot

In [7]:
spectra_df

,Channel,mTurquoise2,mEGFP,cpVenus
0,450,0.000227,0.000111,0.000086
1,459,0.002941,0.000591,0.000098
2,468,0.080711,0.013670,0.000696
3,477,0.115480,0.018401,0.001306
4,486,0.096995,0.028490,0.002480
5,495,0.111672,0.070406,0.003810
6,504,0.109225,0.137999,0.009259
7,512,0.102233,0.179157,0.045444
8,521,0.080510,0.135221,0.145945
9,530,0.067371,0.101641,0.205839


In [8]:
%matplotlib ipympl
fig, ax = plt.subplots(figsize=(3.25, 2.8))
for label in labels:
    plt.plot(spectra_df['Channel'], spectra_df[label])
sns.despine()
plt.grid()
plt.xlabel('Channel')
plt.ylabel('Emission (AU)')
plt.xticks(spectra_df['Channel'][::2], rotation='vertical')
plt.legend(labels)
plt.tight_layout();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save

In [9]:
savechooser = FileChooser(); savechooser

FileChooser(path='C:\Users\dnfst\Jupyter\multicolor roi drawing', filename='', title='HTML(value='', layout=La…

In [10]:
savefiletext = Text(description='Name:', value='emission spectra'); savefiletext

Text(value='emission spectra', description='Name:')

In [11]:
spectra_df.to_csv(join(savechooser.selected_path, savefiletext.value+'.csv'), index=False)